In [72]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import cross_validate, RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler 
from scipy.stats import randint, loguniform

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE, RFECV


In [99]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.options.display.float_format = '{:.6f}'.format

In [74]:
df = pd.read_csv("raw_starting_data.csv")
df

,team,season,corsiPercentage,fenwickPercentage,xGoalsPercentage,xOnGoalFor_PerGame,xGoalsFor_PerGame,xReboundsFor_PerGame,xFreezeFor_PerGame,xPlayStoppedFor_PerGame,xPlayContinuedInZoneFor_PerGame,xPlayContinuedOutsideZoneFor_PerGame,flurryAdjustedxGoalsFor_PerGame,scoreVenueAdjustedxGoalsFor_PerGame,flurryScoreVenueAdjustedxGoalsFor_PerGame,shotsOnGoalFor_PerGame,missedShotsFor_PerGame,blockedShotAttemptsFor_PerGame,shotAttemptsFor_PerGame,goalsFor_PerGame,reboundsFor_PerGame,reboundGoalsFor_PerGame,freezeFor_PerGame,playStoppedFor_PerGame,playContinuedInZoneFor_PerGame,playContinuedOutsideZoneFor_PerGame,savedShotsOnGoalFor_PerGame,savedUnblockedShotAttemptsFor_PerGame,penalityMinutesFor_PerGame,faceOffsWonFor_PerGame,hitsFor_PerGame,takeawaysFor_PerGame,giveawaysFor_PerGame,lowDangerShotsFor_PerGame,mediumDangerShotsFor_PerGame,highDangerShotsFor_PerGame,lowDangerxGoalsFor_PerGame,mediumDangerxGoalsFor_PerGame,highDangerxGoalsFor_PerGame,lowDangerGoalsFor_PerGame,mediumDangerGoalsFor_PerGame,highDangerGoalsFor_PerGame,scoreAdjustedShotsAttemptsFor_PerGame,unblockedShotAttemptsFor_PerGame,scoreAdjustedUnblockedShotAttemptsFor_PerGame,dZoneGiveawaysFor_PerGame,xGoalsFromxReboundsOfShotsFor_PerGame,xGoalsFromActualReboundsOfShotsFor_PerGame,reboundxGoalsFor_PerGame,totalShotCreditFor_PerGame,scoreAdjustedTotalShotCreditFor_PerGame,scoreFlurryAdjustedTotalShotCreditFor_PerGame,xOnGoalAgainst_PerGame,xGoalsAgainst_PerGame,xReboundsAgainst_PerGame,xFreezeAgainst_PerGame,xPlayStoppedAgainst_PerGame,xPlayContinuedInZoneAgainst_PerGame,xPlayContinuedOutsideZoneAgainst_PerGame,flurryAdjustedxGoalsAgainst_PerGame,scoreVenueAdjustedxGoalsAgainst_PerGame,flurryScoreVenueAdjustedxGoalsAgainst_PerGame,shotsOnGoalAgainst_PerGame,missedShotsAgainst_PerGame,blockedShotAttemptsAgainst_PerGame,shotAttemptsAgainst_PerGame,goalsAgainst_PerGame,reboundsAgainst_PerGame,reboundGoalsAgainst_PerGame,freezeAgainst_PerGame,playStoppedAgainst_PerGame,playContinuedInZoneAgainst_PerGame,playContinuedOutsideZoneAgainst_PerGame,savedShotsOnGoalAgainst_PerGame,savedUnblockedShotAttemptsAgainst_PerGame,penalityMinutesAgainst_PerGame,faceOffsWonAgainst_PerGame,hitsAgainst_PerGame,takeawaysAgainst_PerGame,giveawaysAgainst_PerGame,lowDangerShotsAgainst_PerGame,mediumDangerShotsAgainst_PerGame,highDangerShotsAgainst_PerGame,lowDangerxGoalsAgainst_PerGame,mediumDangerxGoalsAgainst_PerGame,highDangerxGoalsAgainst_PerGame,lowDangerGoalsAgainst_PerGame,mediumDangerGoalsAgainst_PerGame,highDangerGoalsAgainst_PerGame,scoreAdjustedShotsAttemptsAgainst_PerGame,unblockedShotAttemptsAgainst_PerGame,scoreAdjustedUnblockedShotAttemptsAgainst_PerGame,dZoneGiveawaysAgainst_PerGame,xGoalsFromxReboundsOfShotsAgainst_PerGame,xGoalsFromActualReboundsOfShotsAgainst_PerGame,reboundxGoalsAgainst_PerGame,totalShotCreditAgainst_PerGame,scoreAdjustedTotalShotCreditAgainst_PerGame,scoreFlurryAdjustedTotalShotCreditAgainst_PerGame,playoff
0,MIN,2008,0.48,0.48,0.48,26.951341,2.292927,1.809512,6.051585,0.879268,15.443049,11.653902,2.225122,2.298780,2.230976,27.536585,11.060976,11.975610,50.573171,2.609756,1.597561,0.304878,5.439024,0.512195,12.146341,16.292683,24.926829,35.987805,10.329268,27.109756,17.902439,5.365854,7.682927,30.353659,6.451220,1.792683,0.887561,0.757927,0.647439,0.902439,0.926829,0.780488,50.710732,38.597561,38.692317,3.743902,0.405244,0.291585,0.293902,2.373049,2.378537,2.328049,29.565854,2.467683,1.993902,6.798049,0.955854,16.666707,12.588902,2.387073,2.463171,2.382683,30.707317,11.573171,12.597561,54.878049,2.402439,1.780488,0.353659,7.548780,0.731707,13.560976,16.256098,28.304878,39.878049,11.207317,28.792683,17.597561,5.402439,7.097561,33.219512,7.158537,1.902439,0.992927,0.835854,0.638902,0.853659,0.853659,0.695122,54.759878,42.280488,42.191707,3.134146,0.442683,0.341341,0.360732,2.495610,2.492439,2.442073,0
1,BOS,2008,0.50,0.50,0.51,29.458889,2.760370,1.976667,6.665309,0.952222,16.346296,12.585556,2.665926,2.789012,2.693704,30.370370,11.123457,12.938272,54.432099,3

In [75]:
df_train = df[df["season"] != 2025]
df_test = df[df["season"] == 2025]
print(df_train.shape, df_test.shape)

(522, 100) (32, 100)


In [76]:
team_key = df_test.loc[:, "team"]
df_train = df_train.drop(columns=["team", "season"])
df_test = df_test.drop(columns=["team", "season"])

In [ ]:
print(df_train.shape, df_test.shape)
team_key

In [78]:
X_train = df_train.drop(columns=["playoff"])
y_train = df_train.loc[:, "playoff"]
X_test = df_test.drop(columns=["playoff"])
y_test = df_test.loc[:, "playoff"]

In [81]:
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

In [84]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

In [87]:
lr = LogisticRegression(max_iter=1000)
rfe_cv = RFECV(lr)
rfe_cv.fit(X_train_scaled, y_train)
print(f"Number of selected features: {rfe_cv.n_features_.item(), X_train.shape[1]}")

Number of selected features: (3, 97)


In [89]:
X_train.columns[rfe_cv.support_]

Index(['goalsFor_PerGame', 'goalsAgainst_PerGame',
       'highDangerGoalsAgainst_PerGame'],
      dtype='object')

In [90]:
X_train_selected = rfe_cv.transform(X_train_scaled)
X_test_selected = rfe_cv.transform(X_test_scaled)

In [91]:
final_lr = LogisticRegression(max_iter=1000)
final_lr.fit(X_train_selected, y_train)
final_lr.score(X_test_selected, y_test)

0.8125

In [92]:
param_choices = {
    "C": loguniform(1e-3, 1e3)
}
random_search = RandomizedSearchCV(final_lr, param_choices, n_iter=100, verbose=1, n_jobs=-1)
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


c:\Users\ryanh\code\tests\nhl-test-2\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,"estimator estimator: estimator objectAn object of that type is instantiated for each grid point.This is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",LogisticRegre...max_iter=1000)
,"param_distributions param_distributions: dict or list of dictsDictionary with parameters names (`str`) as keys and distributionsor lists of parameters to try. Distributions must provide a ``rvs``method for sampling (such as those from scipy.stats.distributions).If a list is given, it is sampled uniformly.If a list of dicts is given, first a dict is sampled uniformly, andthen a parameter is sampled using that dict as above.",{'C': <scipy.stats....00235C5B80E90>}
,"n_iter n_iter: int, default=10Number of parameter settings that are sampled. n_iter tradesoff runtime vs quality of the solution.",100
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.If None, the estimator's score method is used.",None
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given the ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``RandomizedSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",None
,"verbose

In [93]:
pd.DataFrame(random_search.cv_results_)[["mean_test_score", "param_C", "mean_fit_time", "rank_test_score"]].set_index("rank_test_score").sort_index()

,mean_test_score,param_C,mean_fit_time
rank_test_score,,,
1,0.873553,1.210172,0.195291
2,0.873535,4.293747,0.204450
2,0.873535,3.512998,0.176039
4,0.871648,0.164044,0.176777
5,0.871630,2.601846,0.191752
...,...,...,...
96,0.764304,0.001270,0.032999
97,0.762381,0.001203,0.029923
97,0.762381,0.001210,0.051052


In [95]:
final_pipe = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000, C=1.210172))
final_pipe.fit(X_train, y_train)
final_pipe.score(X_test, y_test)

0.78125

In [97]:
np.set_printoptions(suppress=True, precision=4)

In [104]:
playoff_odds_df = pd.DataFrame(final_pipe.predict_proba(X_test), index=team_key).sort_values(1, ascending=False)[[1]]
playoff_odds_df

,1
team,
COL,1.000000
DAL,0.999932
TBL,0.999496
WSH,0.989515
MIN,0.989494
PIT,0.935266
CAR,0.844104
MTL,0.775036
NYI,0.757750


In [42]:
# features = []
# train_score = []
# test_score = []

# for f in range(97, 9, -1):
#     print(f)
#     rfe = RFE(lr, n_features_to_select=f)
#     rfe.fit(X_train, y_train)
    
#     features.append(f)
#     train_score.append(rfe.score(X_train, y_train))
#     test_score.append(rfe.score(X_test, y_test))
    


In [43]:
# results_df = pd.DataFrame({
#     "features": features,
#     "train_score": train_score,
#     "test_score": test_score
# })
# results_df

In [44]:
# fig, ax = plt.subplots(figsize=(15, 5))
# ax.plot(results_df.index, results_df['train_score'], marker='o', linewidth=2, markersize=8, label='Train score')
# ax.plot(results_df.index, results_df['test_score'], marker='o', linewidth=2, markersize=8, label='Test score')
# ax.set_xlabel('Model')
# ax.set_ylabel('Score')
# ax.set_title('Line Chart')
# ax.legend()
# ax.grid(alpha=0.3)

# plt.tight_layout()
# plt.show()